# Virtual experiments, replicating Lahiri and Bevan 2020

In this notebook, we try and replicate a few of the dopamine experiments in Lahiri and Bevan 2020.

4Hz optogenetical stimulation (5 pulses) yielded approximately 0.3 uM DA concentration, while 20Hz stimulation yielded (5 pulses) on average 0.5 uM DA concentration. Here we model the optogenetic activation by setting the DA concentration to these values. In both cases the duration of the resulting DA pulse was set to 5 seconds. 


## Experiment 1

Current injection driving the dSPN to approximately 10Hz, after 5 seconds 0.3 uM DA is added, then current injection is active for another 5 seconds.

Repeat experiment, but this time current injection drives dSPN to approximately 15Hz, and then after 5 seconds 0.5 uM DA is added.

Compare to Figure 1E, 1G, 1H

In [1]:
import os
from snudda import Snudda

neuron = "dspn"
# neuron = "ispn"

neuron_path = os.path.join("data", neuron)
network_path = os.path.join("networks", "bevan_fig1_bath_current_SBML")

In [2]:
snudda = Snudda(network_path=network_path)
si = snudda.init_tiny(neuron_paths=neuron_path, neuron_names=neuron, number_of_neurons=[5], 
                      random_seed=1234)

si.network_data["regions"]["Cube"]["neurons"]["dspn"]["reaction_diffusion"] = "data/JSON/reaction_diffusion_D1.json"
# si.network_data["regions"]["Cube"]["neurons"]["ispn"]["reaction_diffusion"] = "data/JSON/reaction_diffusion_D2.json-updated"


# How the ion channels are modified by DA
# OBS, we include SK direkt modulation, in relality it should be modulated by DA acting on Ca 
# si.network_data["regions"]["Cube"]["neurons"][neuron]["modulation"] = "modulation_parameters.json"
si.network_data["regions"]["Cube"]["neurons"][neuron]["modulation"] = "data/modulation-bevan2020.json"

si.network_data["regions"]["Cube"]["neurons"][neuron]["modulation_key"] = "abc"

si.write_json()

snudda.create_network()

Warning, no snudda_data set!
Adding neurons: dspn from dir data/dspn
Writing networks/bevan_fig1_bath_current_SBML/network-config.json
Writing networks/bevan_fig1_bath_current_SBML/network-config.json
Placing neurons
Network path: networks/bevan_fig1_bath_current_SBML
Reading SNUDDA_DATA=None from networks/bevan_fig1_bath_current_SBML/network-config.json
Reading SNUDDA_DATA=/home/hjorth/HBP/Snudda/snudda/data from networks/bevan_fig1_bath_current_SBML/network-synapses.hdf5
No n_putative_points and putative_density, setting n_putative_points = 102
(this must be larger than the number of neurons you want to place)
Generating 102 points for networks/bevan_fig1_bath_current_SBML/mesh/Cube-cube-mesh-3.9602691983237216e-05.obj
Filtering, keeping inside points: 8 / 44
neuron_name = 'dspn_0', num = 5, neuron_path = 'data/dspn/str-dspn-e150602_c1_D1-mWT-0728MSN01-v20211026'
stop_parallel disabled, to keep pool running.

Execution time: 0.1s
Touch detection
Network path: networks/bevan_fig1_bath

In [2]:
# Free memory
snudda = None

mech_dir = "/home/hjorth/HBP/BasalGangliaData/data/neurons/mechanisms"
sample_dt = None # 0.00005

sim_config_on = os.path.join("data", "bevan_fig1_with_DA_sbml.json")
sim_config_off = os.path.join("data", "bevan_fig1_no_DA_sbml.json")

sim_output_neuromodulation_ON = os.path.join(network_path, "simulation", "output_neuromodulation_ON.hdf5")
sim_output_neuromodulation_OFF = os.path.join(network_path, "simulation", "output_neuromodulation_OFF.hdf5")

sim_time = 8  # 12
n_workers = 5

In [4]:
run_str_on = f"mpirun -n {n_workers} snudda simulate {network_path} --time {sim_time} --simulation_config {sim_config_on} --mechdir {mech_dir} --enable_rxd_neuromodulation"
print(run_str_on)

mpirun -n 5 snudda simulate networks/bevan_fig1_bath_current_SBML --time 8 --simulation_config data/bevan_fig1_with_DA_sbml.json --mechdir /home/hjorth/HBP/BasalGangliaData/data/neurons/mechanisms --enable_rxd_neuromodulation


In [ ]:
os.system(run_str_on)

In [5]:
run_str_off = f"mpirun -n {n_workers} snudda simulate {network_path} --time {sim_time} --simulation_config {sim_config_off} --mechdir {mech_dir} "
print(run_str_off)

mpirun -n 5 snudda simulate networks/bevan_fig1_bath_current_SBML --time 8 --simulation_config data/bevan_fig1_no_DA_sbml.json --mechdir /home/hjorth/HBP/BasalGangliaData/data/neurons/mechanisms 


In [ ]:
os.system(run_str_off)

In [3]:
from snudda.utils import SnuddaLoadSimulation

nd = SnuddaLoadSimulation(sim_output_neuromodulation_ON)
time = nd.get_time()
data_pka = nd.get_data("PKAc", 0)[0][0]
data_da = nd.get_data("DA", 0)[0][0]
data_da_external = nd.get_data("DA", 0)[0][0]

# This is saved with add_rxd_internal_concentration_recording_all -- check that it worked 
data_pka_all0 = nd.get_data("PKAc", 0)[0][0]

Loading networks/bevan_fig1_bath_current_SBML/simulation/output_neuromodulation_ON.hdf5


In [4]:
nd.list_data_types(0)

['AC5',
 'AC5GaolfGTP',
 'AC5GaolfGTP_ATP',
 'AC5_ATP',
 'AMP',
 'ATP',
 'D1R',
 'D1RDA',
 'D1RDAGolf',
 'D1RGolf',
 'DA',
 'GaolfGDP',
 'GaolfGTP',
 'Gbgolf',
 'Golf',
 'PDE10',
 'PDE10_cAMP',
 'PDE10c',
 'PDE10c_cAMP',
 'PDE4',
 'PDE4_cAMP',
 'PKA',
 'PKAc',
 'PKAcAMP2',
 'PKAcAMP4',
 'PKAreg',
 'cAMP',
 'cal12_ms.modulation_factor',
 'cal13_ms.modulation_factor',
 'kaf_ms.modulation_factor_g',
 'kaf_ms.modulation_factor_shift',
 'kas_ms.modulation_factor',
 'kir_ms.modulation_factor',
 'naf_ms.modulation_factor',
 'spikes',
 'voltage']

In [5]:
data_types = nd.list_data_types(0)
all_species_data = nd.get_all_data(neuron_id=0, exclude=["spikes", "voltage"])
time = nd.get_time()
voltage = nd.get_data("voltage", [0])

In [6]:
import plotly.graph_objects as go
import plotly.io as pio
pio.renderers.default = "iframe"  # Do not save plots in the notebook, they can get BIG

fig = go.Figure()
for data_type in all_species_data:
    idx = time >= 0.0
    fig.add_trace(go.Scatter(x=time[idx], y=all_species_data[data_type][0][0].T[0][idx], name=data_type, line={"width":4}))

"""
fig.update_layout(xaxis_title="Time (s)", yaxis_title="Concentration", width=1000, height=800,
                 font={"size":18},  # General font size for all elements
                 legend={"font":{"size":16}},  # Specific font size for legend
                 xaxis={"title":{"font":{"size":20}}, "tickfont":{"size":14}},  # X-axis title and tick labels
                 yaxis={"title":{"font":{"size":20}}, "tickfont":{"size":14}})   # Y-axis title and tick labels
"""
fig.show()

In [7]:
# Reporting plot

import plotly.graph_objects as go
import plotly.io as pio
pio.renderers.default = "iframe"  # Do not save plots in the notebook, they can get BIG

fig = go.Figure()
for data_type in ["D1RDA", "PKAc", "cAMP"]: #all_species_data:
    idx = time >= 0.0
    fig.add_trace(go.Scatter(x=time[idx], y=all_species_data[data_type][0][0].T[0][idx], name=data_type, line={"width":4}))

fig.update_layout(xaxis_title="Time (s)", yaxis_title="Concentration", width=1000, height=800,
                  font={"size":18},  # General font size for all elements
                  legend={"font":{"size":50}},  # Specific font size for legend
                  xaxis={"title":{"font":{"size":40}}, "tickfont":{"size":30}},  # X-axis title and tick labels
                  yaxis={"title":{"font":{"size":40}}, "tickfont":{"size":30}})   # Y-axis title and tick labels


fig.show()

fig.write_image("bevan_1-example-da-cascade-1.png", width=1200, height=800)


fig2 = go.Figure()
for data_type in ["cal12_ms.modulation_factor", "cal13_ms.modulation_factor", 
                  "kas_ms.modulation_factor", "kir_ms.modulation_factor", "naf_ms.modulation_factor", ]: #all_species_data:
    idx = time >= 0.0
    data_type_str = data_type.replace("modulation_factor", "modulation")
    fig2.add_trace(go.Scatter(x=time[idx], y=all_species_data[data_type][0][0].T[0][idx], name=data_type_str, line={"width":4}))

fig2.update_layout(xaxis_title="Time (s)", yaxis_title="Modulation factor", width=1000, height=800,
                  font={"size":18},  # General font size for all elements
                  legend={"font":{"size":50}},  # Specific font size for legend
                  xaxis={"title":{"font":{"size":40}}, "tickfont":{"size":30}},  # X-axis title and tick labels
                  yaxis={"title":{"font":{"size":40}}, "tickfont":{"size":30}})   # Y-axis title and tick labels

fig2.show()
fig2.write_image("bevan_fig1_biochem.png", width=1200, height=800)

nd = None

In [8]:
from snudda.utils import SnuddaLoadSimulation

nd_on = SnuddaLoadSimulation(sim_output_neuromodulation_ON)
data_types = nd_on.list_data_types(0)
all_species_data = nd_on.get_all_data(neuron_id=0, exclude=["spikes", "voltage"])
time_on = nd_on.get_time()
voltage_on = nd_on.get_data("voltage")

Loading networks/bevan_fig1_bath_current_SBML/simulation/output_neuromodulation_ON.hdf5


In [9]:
nd_off = SnuddaLoadSimulation(sim_output_neuromodulation_OFF)
time_off = nd_off.get_time()
voltage_off = nd_off.get_data("voltage")

Loading networks/bevan_fig1_bath_current_SBML/simulation/output_neuromodulation_OFF.hdf5


In [10]:
voltage_off[0]

{0: array([[-0.086     , -0.086     ],
        [-0.08614763, -0.08614763],
        [-0.08624246, -0.08624246],
        ...,
        [-0.05623192, -0.05623192],
        [-0.05622798, -0.05622798],
        [-0.05622404, -0.05622404]]),
 1: array([[-0.086     , -0.086     ],
        [-0.08615903, -0.08615903],
        [-0.08626254, -0.08626254],
        ...,
        [-0.06665854, -0.06665854],
        [-0.06676704, -0.06676704],
        [-0.06687107, -0.06687107]]),
 2: array([[-0.086     , -0.086     ],
        [-0.08615812, -0.08615812],
        [-0.08626215, -0.08626215],
        ...,
        [-0.0619864 , -0.0619864 ],
        [-0.06198573, -0.06198573],
        [-0.06198506, -0.06198506]]),
 3: array([[-0.086     , -0.086     ],
        [-0.0861413 , -0.0861413 ],
        [-0.08623328, -0.08623328],
        ...,
        [-0.05072201, -0.05072201],
        [-0.05071086, -0.05071086],
        [-0.05069967, -0.05069967]]),
 4: array([[-0.086     , -0.086     ],
        [-0.08616114, -0.

In [36]:
import numpy as np
import json
with open("data/bevan_fig1_with_DA_sbml.json", "r") as f:
    config = json.load(f)

da_time = np.array(config["bath_application"]["DA"]["time"])
da_conc = np.array(config["bath_application"]["DA"]["concentration"]) * 1e6


In [37]:
# import plotly.graph_objects as go
import plotly.io as pio
pio.renderers.default = "iframe"  # Do not save plots in the notebook, they can get BIG


for neuron_id in voltage_on[0].keys():
    
    fig = go.Figure()
    
    sct_on = go.Scatter(x=time_on, y=voltage_on[0][neuron_id][:,0], name="DA", opacity=0.5,  line={"width":4, "color":"red"}, yaxis="y1")
    sct_off = go.Scatter(x=time_off, y=voltage_off[0][neuron_id][:,0], name="No DA", opacity=0.5,  line={"width":4, "color":"black"}, yaxis="y1")

    da_curve = go.Scatter(x=da_time, y=da_conc, opacity=0.2, name="DA conc", line={"width":0, "color": "rgba(255, 0, 0, 0.2)"},mode="lines",  
                          yaxis="y2",fill="tozeroy", fillcolor="rgba(255, 0, 0, 0.2)")
    
    fig.add_traces([sct_on, sct_off, da_curve])

    fig.update_layout(
        xaxis_title="Time (s)", 
        yaxis_title="Voltage (V)",
        width=1000, height=800,
        font={"size":18},
        legend={"font":{"size":30}},  # Increased size slightly for readability
        xaxis={"title":{"font":{"size":40}}, "tickfont":{"size":30}, "range":[0,8]},
        yaxis=dict(  # Primary y-axis (Voltage)
            title="Voltage (V)",
            titlefont={"size":40},
            tickfont={"size":30},
            side="left"
        ),
        yaxis2=dict(  # Secondary y-axis (DA)
            title="DA Conc (nM)",
            titlefont={"size":30},
            tickfont={"size":20},
            overlaying="y",
            side="right",
            range=[0, max(da_conc)*10],
            tickvals=[0, max(da_conc)],
            ticktext=["0", f"{max(da_conc)}"],
            showgrid=False
        )
    )

    fig.show()
    fig.write_image(f"bevan-figure-{neuron_id}-volt-trace.png", scale=2, height=800, width=1200)

In [16]:
from snudda.plotting import SnuddaPlotSpikeRaster2

network_file = os.path.join(network_path, "network-synapses.hdf5")
spr_on = SnuddaPlotSpikeRaster2(network_path=network_path, network_file=network_file, simulation_file=sim_output_neuromodulation_ON)

network_file = os.path.join(network_path, "network-synapses.hdf5")
spr_off = SnuddaPlotSpikeRaster2(network_path=network_path, network_file=network_file, simulation_file=sim_output_neuromodulation_OFF)

time_ranges = [(2,3), (6,7)]
freq_on = spr_on.snudda_simulation_load.get_frequency(neuron_id=[0, 1, 2, 3], time_ranges=time_ranges)
freq_off = spr_off.snudda_simulation_load.get_frequency(neuron_id=[0, 1, 2, 3], time_ranges=time_ranges)



Loading networks/bevan_fig1_bath_current_SBML/simulation/output_neuromodulation_ON.hdf5
Loading networks/bevan_fig1_bath_current_SBML/simulation/output_neuromodulation_OFF.hdf5


In [18]:
freq_on, freq_off

(array([[ 8., 28.],
        [11., 32.],
        [ 6., 25.],
        [ 7., 38.]]),
 array([[ 8.,  9.],
        [11., 11.],
        [ 6.,  7.],
        [ 7.,  6.]]))

In [49]:
def make_freq_plot(data, title):

    fig = go.Figure()

    # Add one line per row
    for row in data:
        fig.add_trace(go.Scatter(
            x=[0, 1],
            y=row,
            mode='lines+markers',
            line=dict(color='gray', width=2),
            marker=dict(size=10, color='black'),
            showlegend=False
        ))

    # Mean line
    mean_vals = np.mean(data, axis=0)
    fig.add_trace(go.Scatter(
        x=[0, 1],
        y=mean_vals,
        mode='lines+markers',
        line=dict(color='black', width=6),
        marker=dict(size=12, color='black'),
        name='Mean'
    ))

    # Layout
    fig.update_layout(
        title=title,
        xaxis=dict(
            tickmode='array',
            tickvals=[0, 1],
            ticktext=["t=(2,3)", "t=(6,7)"],
            title='',
            tickfont=dict(size=18),
            titlefont=dict(size=20)
        ),
        yaxis=dict(
            title='Frequency (Hz)',
            titlefont=dict(size=20),
            tickfont=dict(size=18)
        ),
        width=300,
        height=500,
        font=dict(size=16)
    )
    return fig



In [50]:
# Create and show plots
fig_on = make_freq_plot(freq_on, "With Dopamine")
fig_on.show()
fig_on.write_image("fig1_frequency_change_with_da.png", scale=2)

In [52]:
fig_off = make_freq_plot(freq_off, "Without Dopamine")
fig_off.show()
fig_off.write_image("fig1_frequency_change_no_da.png", scale=2)

## Experiment 2

Can we show persistence for 15 minutes? (Compare Figure 2)

## Experiment 3

Current injection for 3 seconds, trigger 2 (4) spikes in dSPN. After DA application (1 second, 0.5u M) spikes come earlier and at a higher frequency.

Compare Figure 4 B, C, D

## Experiment 4

Subthreshold current injection (<10 mV depolarisation). Size of depolarisation not affected by DA.

Suprathreshold current injection, with DA lower threshold, and faster spiking.

Compare Figure 5

In [ ]:
Experiment 5

Compare Figure 6